## EDA Part 1

### Load credentials from Project root

In [1]:
import os
from dotenv import load_dotenv

# Load .env from project root
load_dotenv()  # looks for .env in the current working directory

AWS_ACCESS_KEY_ID = os.getenv("AWS_Acess_Key_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_Secret_Access_Key")
AWS_REGION = os.getenv("AWS_REGION", "us-east-1")

ATHENA_S3_OUTPUT = os.getenv("ATHENA_S3_OUTPUT")
ATHENA_DATABASE = os.getenv("ATHENA_DATABASE")

AWS_ACCESS_KEY_ID[:4], AWS_REGION, ATHENA_DATABASE


('AKIA', 'us-east-1', 's3_atmos_stclair')

### Load the data by creating boto session using awswrangler

In [2]:
import boto3
import awswrangler as wr
import pandas as pd

session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_REGION,
)


### All the Tables in the Athena Database

In [3]:
df_tables = wr.catalog.tables(database=ATHENA_DATABASE, boto3_session=session)
table_names = df_tables["Table"].tolist()

print("Table names:", table_names)
print("Number of tables:", len(table_names))

Table names: ['achievement_user_completions', 'achievements', 'banner_clicks', 'bdg_user_completions', 'certification_users', 'comment_likes', 'community_comments', 'country_states', 'instances', 'instances_v', 'li_attempts', 'li_stats_agg_all', 'li_stats_agg_all_v', 'mi_user_mission_step_completions', 'offboarded_instances', 'poll_user_completions', 'rank_user_completions', 'rcs_historic_users', 'rcs_historic_users_v', 'rcs_lifetime_stats_by_month_v', 'rcs_user_lifetimes_no_stats_v', 'reward_prizes', 'reward_user_claims', 'reward_user_claims_v', 'rewards', 'sale_user_submissions', 'snippet_view_logs', 'snippets', 'survey_user_completions', 'test2_v', 'test_v', 'users']
Number of tables: 32


In [4]:
df_tables

,Database,Table,Description,TableType,Columns,Partitions
0,s3_atmos_stclair,achievement_user_completions,,EXTERNAL_TABLE,"instance_id, generated_for_date, generated_ts,...",
1,s3_atmos_stclair,achievements,,EXTERNAL_TABLE,"instance_id, generated_for_date, generated_ts,...",
2,s3_atmos_stclair,banner_clicks,,EXTERNAL_TABLE,"instance_id, generated_for_date, generated_ts,...",
3,s3_atmos_stclair,bdg_user_completions,,EXTERNAL_TABLE,"instance_id, generated_for_date, generated_ts,...",
4,s3_atmos_stclair,certification_users,,EXTERNAL_TABLE,"instance_id, generated_for_date, generated_ts,...",
5,s3_atmos_stclair,comment_likes,,EXTERNAL_TABLE,"instance_id, generated_for_date, generated_ts,...",
6,s3_atmos_stclair,community_comments,,EXTERNAL_TABLE,"instance_id, generated_for_date, generated_ts,...",
7,s3_atmos_stclair,country_states,,EXTERNAL_TABLE,"country_short_code, country_name, state_name, ...",
8,s3_atmos_stclair,instances,,EXTERNAL_TABLE,"instance_id, client_name, site_name, site_url,...",
9,s3_atmos_stclair,instances_v,,VIRTUAL_VIEW,"instance_id, client_name, region_name, site_na...",


In [5]:
query = "SELECT * FROM rcs_historic_users"  

df_historic_users = wr.athena.read_sql_query(
    sql=query,
    database=ATHENA_DATABASE,
    s3_output=ATHENA_S3_OUTPUT,
    boto3_session=session,
    ctas_approach=False,
    use_threads=True,
)



MemoryError: 